### Distribución Espacio-temporal de las Descargas Eléctricas
***
Facultad de Ingeniería, Universidad Nacional de Asunción<br>
Laboratorio de Mecánica y Energía<br>
Est. Adriana A. Oviedo R.
***
##### Ubicación: Paraguay
*Coordenadas del área de estudio*
>-  -18,-64
>-  -18,-54
>-  -28,-64
>-  -28,-54
<img src="attachment:image.png" width="500">
##### Periodo: 2016 - 2020
##### Tipos de datos: Pulsos de descargas eléctricas
*Datos de la Dirección de Meterología e Hidrología - Paraguay*
* El timestamp de los datos indica el tiempo en el que ocurre el pulso de mayor amplitud del rayo
* Clasificación: Type
    -  "0": Cloud-to-Ground -> Descarga nube a tierra.
    -  "1": Intra-Cloud -> Descarga entre nubes; podrían indicar también otros tipos distintos de 
    las descargas CG.
    -  "9": Keep-Alive, función de los sensores para controlar su funcionamiento.
    -  "40": Provienen de otra institución, distinta a ENTLN.
* Altura: solamente indica la altura de las descargas tipo 1.
* Coordenadas: Latitud - Longitud (grados decimales)
* Polaridad: signo de la corriente máxima generada por la descarga
* Amplitud: magnitud de la corriente máxima generada por la descarga<br>
 *Datos de los mapas obtenidos del Instituto Nacional de Estadística*, descargables en:
> https://www.ine.gov.py/microdatos/cartografia-digital-2012.php
- Tipos de archivos: .shp (shapefiles)
- No posee solo contorno del Paraguay.

Para la primera parte del análisis, haremos uso de los datos de la Dirección de Meteorología e Hidrología, estos recortados respecto al contorno geográfico del Paraguay. 
* Distribución Anual
* Distribución Estacional
* Distribución Mensual
* Distribución Diaria

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

import geopandas as gpd
import pandas as pd
import earthpy as et
import seaborn as sns
from scipy import ndimage
from shapely.geometry import Point
from shapely.geometry import box
from pandas.plotting import register_matplotlib_converters
import warnings

register_matplotlib_converters()

warnings.filterwarnings('ignore', 'Geoseries.notna', UserWarning)

#Configuración de gráficos
sns.set(font_scale = 1.5, style = 'whitegrid')
#Configuración de directorio de trabajo
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

###### Acceso a archivos .shp que contienen el contorno del mapa del país y sus departamentos.
`mapa = gpd.read_file(mapa_ubi)`

In [2]:
mes = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
       'julio', 'agosto', 'setiembre', 'octubre', 'noviembre', 'diciembre']
mes_code = dict(zip(mes,['01','02','03','04','05','06',
                     '07','08', '09','10','11','12']))
dia = []
for f in range(1,32):
    dia.append(f)
bis = dict(zip(mes,[31,29,31,30,31,30,31,31,30,31,30,31]))
nor = dict(zip(mes,[31,28,31,30,31,30,31,31,30,31,30,31]))
dia_mes = dict(zip(['b','n'],[bis,nor]))
#Ubicación de los shapefiles
pais_path = os.path.join('data','earthpy-downloads',
                        'rayos-paraguay','cartografia',
                        'contorno_Paraguay.shp')
dptos_path = os.path.join('data','earthpy-downloads',
                         'rayos-paraguay','cartografia',
                         'PAIS','Departamentos_Paraguay.shp')

#Uso de gpd para abrir .shp
pais = gpd.read_file(pais_path)
dptos = gpd.read_file(dptos_path)

#Centroides de los polígonos que forman los departamentos, para
#tener los nombres de los mismos y visualizarlos en el mapa
dptos_nombres = gpd.GeoDataFrame(dptos,crs = pais.crs)
dptos_nombres = dptos_nombres.assign(geometry = dptos_nombres['geometry'].centroid)

C:\Users\AgOvi\AppData\Local\Temp\ipykernel_12052\994893187.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dptos_nombres = dptos_nombres.assign(geometry = dptos_nombres['geometry'].centroid)


<div class="alert alert-block alert-info">
<b>Tip:</b> Utilizando listas, creamos listas anidadas con las ubicaciones de los archivos para más orden. Luego, utilizamos diccionarios para acceder por mes o por día a los archivos.</div>

In [3]:
def abrir_archivos(year):
    '''
    year debe ser un str que represente el año al que se quiere acceder.
    La función devuelve un dict que contiene los GeoDataFrame.
    df = abrir_archivos('año')[0]
    err = abrir_archivos('año')[1]
    b = df['mes'][dia]
    Por ahora esta funcion solo se debe utilizar para el periodo 2016-2019.
    2020 - 2022 presenta otro formato en las columnas del archivo .csv
    
    '''
    print('Leyendo: ', year, '...')
    mes = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
      'julio', 'agosto', 'setiembre', 'octubre', 'noviembre', 'diciembre']
    mes_code = dict(zip(mes,['01','02','03','04','05','06',
                         '07','08', '09','10','11','12']))
    bis = dict(zip(mes,[31,29,31,30,31,30,31,31,30,31,30,31]))
    nor = dict(zip(mes,[31,28,31,30,31,30,31,31,30,31,30,31]))
    dia_mes = dict(zip(['b','n'],[bis,nor]))
    dia = []
    for f in range(1,32):
        dia.append(f)
    if (year != '2019')&(year != '2020')&(year!='2021'):
        path_init = os.path.join('data', 'earthpy-downloads', 'rayos-paraguay',
                       'dmh', str(year)+'\\')
        list1 = [path_init + f for f in os.listdir(path_init) if f.startswith(year+'_')]
        list2 = []
        for i in range(len(list1)):
            list2.append([list1[i]+ '\\' +
                          f for f in os.listdir(list1[i]) if f.startswith('Paraguay_')])

        for i in range(len(list2)):
            list2[i] = dict(zip(dia,list2[i]))
        path = 0
        path = dict(zip(mes,list2))
        desc = {}
        err = {}
        cont_e = 0
        cont_d = 0
        for i in path.keys():
            aux = []
            aux2 = []
            coord = 0
            for j in path[i].keys():
                print('Leyendo: ',j, ' de ',i)
                try:
                    aux.append(pd.read_csv(path[i][j], parse_dates=['timestamp'],
                                           index_col=['timestamp'], 
                                           delimiter = ',', engine = 'python',
                                           on_bad_lines = 'warn'))
                    aux[j-1].index = pd.to_datetime(aux[j-1].index,
                                                    infer_datetime_format=True)
                    coord = gpd.points_from_xy(aux[j-1].longitude, aux[j-1].latitude)
                    aux[j-1] = gpd.GeoDataFrame(data = aux[j-1],
                                                geometry = coord, crs = pais.crs)
                    aux[j-1] = gpd.clip(aux[j-1], pais)
                    cont_d+=1
                except Exception as e:
                    cont_e+=1
                    aux.append(gpd.GeoDataFrame())
                    aux2.append(path[i][j])
                    print(e, path[i][j])
                    continue
            desc[i] = dict(zip(dia,aux))
            err[i] = aux2
        print(cont_e, ' archivos vacíos')
        print(cont_d, ' archivos leídos con éxito')
        
        return [desc, err]
    
    elif (year == '2020') | (year == '2021'):
        path_init = os.path.join('data', 'earthpy-downloads', 'rayos-paraguay',
               'dmh', year+'\\')
        list1 = [path_init + f for f in os.listdir(path_init) if f.startswith(year+'_')]
        list2 = []
        for i in range(len(list1)):
            list2.append([list1[i]+ '\\' +
                          f for f in os.listdir(list1[i]) if f.startswith('paraguay_')])

        if year == '2020':
            list2 = dict(zip(mes[2:],list2))
        else:
            list2 = dict(zip(mes, list2))
            
        desc = {}
        for i in list2.keys():
            aux = []
            print(i)
            for j in range(len(list2[i])):
                try:
                    a = pd.read_csv(list2[i][j], parse_dates=['LightningTime'],index_col=['LightningTime'],
                                       delimiter = ',', engine = 'python', on_bad_lines = 'warn')
                    a.index = pd.to_datetime(a.index, infer_datetime_format= True)
                    coord = gpd.points_from_xy(a.Longitude, a.Latitude)
                    a = gpd.GeoDataFrame(data = a, geometry = coord, crs = pais.crs)
                    a = gpd.clip(a, pais)
                    aux.append(a)
                except Exception as e:
                    
                    print(e,'\n', list2[i][j])
                    continue
            desc[i] = aux
                
        desc20 = {}
        print('Concatenando por dia...')
        for i in desc.keys():
            print(i)
            aux = []
            for j in range(1, dia_mes['n'][i]+1):
                print(j)
                df = gpd.GeoDataFrame()
                for k in desc[i]:
                    df = pd.concat([df, k[(k.index.month == int(mes_code[i])) & (k.index.day == j)]],
                                  axis = 0)
                aux.append(df)
            desc20[i] = dict(zip(dia,aux))
            err = 0
        return [desc20, err]
    else:
        path_init = os.path.join('data', 'earthpy-downloads', 'rayos-paraguay',
                           'dmh', '2019\\')
        list1 = [path_init + f for f in os.listdir(path_init) if f.startswith('output_')]
        arch = []
        for i in range(1, len(list1)+1):
            arch.append(str(i))
        arch.sort()
        list_arc = dict(zip(arch, list1))
        dia = []
        for f in range(1,32):
            dia.append(f)
        desc = {}
        print('2019 lleva más tiempo...\n','Favor esperar...')
        for i in list_arc.keys():
            try:
                desc[i] = pd.read_csv(list_arc[i], parse_dates=['timestamp'],index_col=['timestamp'],
                              delimiter = ',', engine = 'python', on_bad_lines = 'warn')
                desc[i].index = pd.to_datetime(desc[i].index, infer_datetime_format = True)
                coord = gpd.points_from_xy(desc[i].longitude, desc[i].latitude)
                desc[i] = gpd.GeoDataFrame(data = desc[i],geometry = coord, crs = pais.crs)
                desc[i] = gpd.clip(desc[i], pais)
            except Exception as e:
                print('\t error: ', i)
        print('SIGUE\n')
        desc19 = {}
        for i in mes:
            print(i)
            aux = []
            for j in range(1, dia_mes['n'][i]+1):
                print('\t',j, i)
                df = gpd.GeoDataFrame()
                for k in range(1,860):
                    a = desc[str(k)][(desc[str(k)].index.month == int(mes_code[i])) & (desc[str(k)].index.day == j)]
                    df = pd.concat([df,a], axis = 0)
                    a = 0
                aux.append(df)
            desc19[i] = dict(zip(dia,aux))
            err = 0
        return [desc19,err]

In [4]:
def plot_map(year, month, day, save=None):
    '''
    Genera los puntos de los rayos ubicados respecto al mapa del Paraguay.
    Si se agrega el parámetro save = True, no muestra el gráfico pero lo 
    guarda en la dirección que indica.
    
    '''
    x = year
    y = month
    z = day
    
    try:
        f, ax = plt.subplots(figsize=(20,20))
        pais.plot(color = 'white', edgecolor = 'k',
                 ax = ax, legend = True)
        dptos.plot(ax = ax, alpha = .5,
                  edgecolor = 'k', color = 'white')
        dfs[x][y][z].plot(ax = ax, column = 'type',
                         markersize = 2, cmap = 'jet',
                         categorical = True, legend = True)
        dptos_nombres.plot(markersize=10, color='white', ax=ax)
        dptos_nombres.apply(lambda x: ax.annotate(text=x['DPTO_DESC'],
                                                  xy = x.geometry.coords[0], 
                                                  xytext=(-25,0), 
                                                  textcoords = 'offset points'),
                            axis = 1)
        f.suptitle('\nDescargas Eléctricas - Paraguay\n'+str(z)+'/'+mes_code[str(y)]+'/'+str(x),
                  fontsize = 24)
        if save:
            plt.savefig(os.path.join('data', 'earthpy-downloads','rayos-paraguay','dist espacial',
                                     str(x), mes_code[y], str(z)+'_'+mes_code[y]+'_'+str(x)+'.png'),
                        facecolor = 'white', bbox_inches = 'tight', dpi = 150)
            plt.close(f)
            plt.show()
        else:
            plt.show()
    except Exception as e:
        plt.close(f)
        print('Esta fecha no tiene datos\n', e)

In [24]:
def dist_mensual(year, dfs):
    '''
    Hace resample de los datos de un año entero, y los agrupa por mes
    '''
    df = gpd.GeoDataFrame()
    for i in dfs[year].keys():
        for j in dfs[year][i].keys():
            try:
                dfs = pd.concat([df, dfs[year][i][j]],
                               axis = 0)
            except:
                df = pd.concat([df, gpd.GeoDataFrame()],
                              axis = 0)
    df.sort_index(inplace = True)
    if int(year) > 2019:
        df = df.resample('M').Amplitude.count()
    else:
        df = df.resample('M').peakcurrent.count()
    
    return df

In [6]:
years = ['2016','2017','2018','2019','2020','2021']
dfs = {}
for i in years:
    dfs[i] = abrir_archivos(i)[0]

Leyendo:  2016 ...
Leyendo:  1  de  enero
Leyendo:  2  de  enero
Leyendo:  3  de  enero
Leyendo:  4  de  enero
Leyendo:  5  de  enero
Leyendo:  6  de  enero
Leyendo:  7  de  enero
Leyendo:  8  de  enero
Leyendo:  9  de  enero
Leyendo:  10  de  enero
Leyendo:  11  de  enero
Leyendo:  12  de  enero
Leyendo:  13  de  enero
Leyendo:  14  de  enero
Leyendo:  15  de  enero
Leyendo:  16  de  enero
Leyendo:  17  de  enero
Leyendo:  18  de  enero
Leyendo:  19  de  enero
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_01\Paraguay_pulse20160119.csv
Leyendo:  20  de  enero
Leyendo:  21  de  enero
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_01\Paraguay_pulse20160121.csv
Leyendo:  22  de  enero
Leyendo:  23  de  enero
Leyendo:  24  de  enero
Leyendo:  25  de  enero
Leyendo:  26  de  enero
Leyendo:  27  de  enero
Leyendo:  28  de  enero
Leyendo:  29  de  enero
Leyendo:  30  de  enero
Leyendo:  31  de  enero
Leyendo:  1  de  febre

Skipping line 172205: Expected 10 fields in line 172205, saw 11


Leyendo:  20  de  marzo
Leyendo:  21  de  marzo
Leyendo:  22  de  marzo
Leyendo:  23  de  marzo
Leyendo:  24  de  marzo
Leyendo:  25  de  marzo
Leyendo:  26  de  marzo
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_03\Paraguay_pulse20160326.csv
Leyendo:  27  de  marzo
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_03\Paraguay_pulse20160327.csv
Leyendo:  28  de  marzo
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_03\Paraguay_pulse20160328.csv
Leyendo:  29  de  marzo
Leyendo:  30  de  marzo
Leyendo:  31  de  marzo
Leyendo:  1  de  abril
Leyendo:  2  de  abril
Leyendo:  3  de  abril
Leyendo:  4  de  abril
Leyendo:  5  de  abril
Leyendo:  6  de  abril
Leyendo:  7  de  abril
Leyendo:  8  de  abril
Leyendo:  9  de  abril
Leyendo:  10  de  abril
Leyendo:  11  de  abril
Leyendo:  12  de  abril
Leyendo:  13  de  abril
Leyendo:  14  de  abril
Leyendo:  15  de  abril
Leyendo:  16  de  abril
N

Skipping line 101968: Expected 10 fields in line 101968, saw 14


Leyendo:  29  de  mayo
Leyendo:  30  de  mayo
Leyendo:  31  de  mayo
Leyendo:  1  de  junio
Leyendo:  2  de  junio
Leyendo:  3  de  junio
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_06\Paraguay_pulse20160603.csv
Leyendo:  4  de  junio
Leyendo:  5  de  junio
Leyendo:  6  de  junio
Leyendo:  7  de  junio
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_06\Paraguay_pulse20160607.csv
Leyendo:  8  de  junio
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_06\Paraguay_pulse20160608.csv
Leyendo:  9  de  junio
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_06\Paraguay_pulse20160609.csv
Leyendo:  10  de  junio
Leyendo:  11  de  junio
Leyendo:  12  de  junio
Leyendo:  13  de  junio
No columns to parse from file data\earthpy-downloads\rayos-paraguay\dmh\2016\2016_06\Paraguay_pulse20160613.csv
Leyendo:  14  de  junio
No columns to parse from file data\earthpy-d

...

No columns to parse from file
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 'LightningTime'
Missing column provided to 'parse_dates': 

C:\Users\AgOvi\miniconda3\envs\earth-analytics-python\lib\site-packages\pygeos\measurement.py:103: RuntimeWarning: invalid value encountered in bounds
  return lib.bounds(geometry_arr, out=out, **kwargs)


Given date string not likely a datetime.
No columns to parse from file


C:\Users\AgOvi\miniconda3\envs\earth-analytics-python\lib\site-packages\pygeos\measurement.py:103: RuntimeWarning: invalid value encountered in bounds
  return lib.bounds(geometry_arr, out=out, **kwargs)


Concatenando por dia...
enero
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
febrero
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
marzo
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
abril
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
mayo
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
junio
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
julio
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
agosto
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
setiembre
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
octubre
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
noviembre
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2

In [ ]:
df_mensual = {}
colors = ['lightblue','green','gold','blueviolet','orchid','darkorange']
path_save = os.path.join('data','earthpy-downloads','rayos-paraguay',
                        'dist espacial','dist_meses.png')
for i in dfs.keys():
    print(i)
    df_mensual[i] = dist_mensual(i)

In [ ]:
##Gráfico temporal - mensual            
f, ax = subplots(len(df_mensual.keys()), figsize = (20,20))
for i in range(len(df_mensual.keys())):
    col = colors[i]
    ax[i].bar(df_mensual[i].index.month.values,
             df_mensual, color = col)
    ax[i].set(title = 'Año '+ df_mensual.keys[i],
             xlabel = 'Mes',
             ylabel = 'Cant. de descargas')
    loc = ticker.MaxNLocator(nbins = 13)
    ax[i].xaxis.set_major_locator(loc)
    for index, value in enumerate(df_mensual[df_mensual.keys()[i]]):
        plt.text(index+1,value, str(value),
                horizontalalignment = 'center')
for a in ax:
    a.label_outer()
f.suptitle('Distribución mensual de las Descargas Eléctricas\nParaguay 2016-2021')
plt.savefig(path_save, facecolor = 'white', dpi = 150)
plt.show()

In [ ]:
def dist_anual(year):
    df = gpd.GeoDataFrame()
    for i in dfs[year].keys():
        for j in dfs[year][i].keys():
            df = pd.concat([df,dfs[year][i][j]],
                          axis = 0)
    if int(year) < 2020:
        df = df.resample('Y').peakcurrent.count()
    else:
        df = df.resample('Y').Amplitude.count()
    
    return df           

In [29]:
dfs['2021']['agosto']

{1: Empty GeoDataFrame
 Columns: [Latitude, Longitude, Height, FlashType, Amplitude, geometry]
 Index: [],
 2: Empty GeoDataFrame
 Columns: [Latitude, Longitude, Height, FlashType, Amplitude, geometry]
 Index: [],
 3: Empty GeoDataFrame
 Columns: [Latitude, Longitude, Height, FlashType, Amplitude, geometry]
 Index: [],
 4: Empty GeoDataFrame
 Columns: [Latitude, Longitude, Height, FlashType, Amplitude, geometry]
 Index: [],
 5:                                Latitude  Longitude   Height  FlashType  \
 LightningTime                                                            
 2021-08-05 19:18:26.258015402 -24.15981  -59.65662      0.0        0.0   
 2021-08-05 19:18:26.283033000 -24.14540  -59.63280      0.0        0.0   
 2021-08-05 19:18:26.373793000 -24.07060  -59.66000      0.0        0.0   
 2021-08-05 19:20:56.895585000 -24.17770  -59.66790      0.0        0.0   
 2021-08-05 19:20:56.799675560 -24.17515  -59.61595  17724.0        1.0   
 2021-08-05 19:20:56.811965000 -24.14230  -5